## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Anloga,GH,5.7947,0.8973,87.80,70,20,11.50,few clouds,2021-01-29 17:00:16
1,1,Tual,ID,-5.6667,132.7500,83.28,75,100,17.83,overcast clouds,2021-01-29 16:56:21
2,2,Lithgow,AU,-33.4833,150.1500,62.01,43,100,3.09,overcast clouds,2021-01-29 17:00:16
3,3,Busselton,AU,-33.6500,115.3333,64.00,80,0,1.99,clear sky,2021-01-29 17:00:17
4,4,Cap Malheureux,MU,-19.9842,57.6142,81.00,78,2,5.01,clear sky,2021-01-29 17:00:17


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Anloga,GH,5.7947,0.8973,87.80,70,20,11.50,few clouds,2021-01-29 17:00:16
1,1,Tual,ID,-5.6667,132.7500,83.28,75,100,17.83,overcast clouds,2021-01-29 16:56:21
4,4,Cap Malheureux,MU,-19.9842,57.6142,81.00,78,2,5.01,clear sky,2021-01-29 17:00:17
8,8,Daru,PG,-9.0763,143.2092,77.95,88,99,14.23,moderate rain,2021-01-29 17:00:18
9,9,Nouadhibou,MR,20.9310,-17.0347,87.80,20,0,12.66,clear sky,2021-01-29 17:00:18
15,15,East London,ZA,-33.0153,27.9116,75.20,73,20,6.91,few clouds,2021-01-29 17:00:19
19,19,Cape Town,ZA,-33.9258,18.4232,75.99,53,20,18.41,few clouds,2021-01-29 16:58:00
22,22,Avarua,CK,-21.2078,-159.7750,77.00,94,20,5.75,few clouds,2021-01-29 16:56:27
28,28,Capanema,BR,-1.1958,-47.1808,83.46,82,46,6.76,scattered clouds,2021-01-29 17:00:23
29,29,Ciudad Bolivar,VE,8.1222,-63.5497,88.00,48,54,10.31,broken clouds,2021-01-29 17:00:23


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_hotel_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_hotel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Anloga,GH,87.80,few clouds,5.7947,0.8973,
1,Tual,ID,83.28,overcast clouds,-5.6667,132.7500,
4,Cap Malheureux,MU,81.00,clear sky,-19.9842,57.6142,
8,Daru,PG,77.95,moderate rain,-9.0763,143.2092,
9,Nouadhibou,MR,87.80,clear sky,20.9310,-17.0347,
15,East London,ZA,75.20,few clouds,-33.0153,27.9116,
19,Cape Town,ZA,75.99,few clouds,-33.9258,18.4232,
22,Avarua,CK,77.00,few clouds,-21.2078,-159.7750,
28,Capanema,BR,83.46,scattered clouds,-1.1958,-47.1808,
29,Ciudad Bolivar,VE,88.00,broken clouds,8.1222,-63.5497,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Anloga,GH,87.80,few clouds,5.7947,0.8973,Pin Drop Hotel
1,Tual,ID,83.28,overcast clouds,-5.6667,132.7500,Grand Vilia Hotel
4,Cap Malheureux,MU,81.00,clear sky,-19.9842,57.6142,Pereybere Hotel & Spa
8,Daru,PG,77.95,moderate rain,-9.0763,143.2092,Daru Lodge
9,Nouadhibou,MR,87.80,clear sky,20.9310,-17.0347,El Medina
...,...,...,...,...,...,...,...
685,Kokoda,PG,75.87,overcast clouds,-8.8785,147.7407,Daito Guest House
686,Constancia,MX,78.01,clear sky,24.2231,-107.1869,Mi casa
687,Belmonte,BR,86.00,broken clouds,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
689,Maningrida,AU,77.38,overcast clouds,-12.0667,134.2667,Dhukurrdji Lodge


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))